In [28]:
import pandas as pd
restaurants_new = pd.read_csv('restaurants_L2_dirty.csv', index_col=0)
restaurants_new.head()


,name,addr,city,phone,type
0,kokomo,6333 w. third st.,la,2139330773,american
1,feenix,8358 sunset blvd. west,hollywood,2138486677,american
2,parkway,510 s. arroyo pkwy .,pasadena,8187951001,californian
3,r-23,923 e. third st.,los angeles,2136877178,japanese
4,gumbo,6333 w. third st.,la,2139330358,cajun/creole


In [29]:
restaurants = pd.read_csv('restaurants_L2.csv',index_col=0)
restaurants.head()

,name,addr,city,phone,type
0,arnie morton's of chicago,435 s. la cienega blv .,los angeles,3102461501,american
1,art's delicatessen,12224 ventura blvd.,studio city,8187621221,american
2,campanile,624 s. la brea ave.,los angeles,2139381447,american
3,fenix,8358 sunset blvd. west,hollywood,2138486677,american
4,grill on the alley,9560 dayton way,los angeles,3102760615,american


In [30]:
restaurants.rename(columns={'name': 'rest_name', 'addr': 'rest_addr','type':'cuisine_type'}, inplace=True)
restaurants_new.rename(columns={'name': 'rest_name', 'addr': 'rest_addr','type':'cuisine_type'}, inplace=True)

In [31]:

# Import process from thefuzz
from thefuzz import process

# Store the unique values of cuisine_type in unique_types
unique_types = restaurants['cuisine_type'].unique()

# Calculate similarity of 'asian' to all values of unique_types
print(process.extract('asian', unique_types, limit = len(unique_types)))

# Calculate similarity of 'american' to all values of unique_types
print(process.extract('american', unique_types, limit = len(unique_types)))

# Calculate similarity of 'italian' to all values of unique_types
print(process.extract('italian', unique_types, limit = len(unique_types)))

[('asian', 100), ('italian', 67), ('american', 62), ('mexican', 50), ('cajun', 40), ('southwestern', 36), ('southern', 31), ('coffeebar', 26), ('steakhouses', 25)]
[('american', 100), ('mexican', 80), ('cajun', 68), ('asian', 62), ('italian', 53), ('southwestern', 41), ('southern', 38), ('coffeebar', 24), ('steakhouses', 21)]
[('italian', 100), ('asian', 67), ('mexican', 43), ('american', 40), ('cajun', 33), ('southern', 27), ('southwestern', 26), ('steakhouses', 26), ('coffeebar', 12)]


In [32]:
# Iterate through categories
categories = restaurants['cuisine_type'].unique()
# categories = ['italian', 'asian',  'american']
for cuisine in categories:  
  # Create a list of matches, comparing cuisine with the cuisine_type column
  matches = process.extract(cuisine, restaurants['cuisine_type'], limit=len(restaurants.cuisine_type))

  # Iterate through the list of matches
  for match in matches:
     # Check whether the similarity score is greater than or equal to 80
    if match[1] >= 80:
      # If it is, select all rows where the cuisine_type is spelled this way, and set them to the correct cuisine
      restaurants.loc[restaurants['cuisine_type'] == match[0]] = cuisine
      
# Inspect the final result
print(restaurants['cuisine_type'].unique())

['mexican' 'asian' 'italian' 'coffeebar' 'southern' 'steakhouses' 'cajun']


In [33]:
import pandas as pd
import recordlinkage
# Create an indexer and object and find possible pairs
indexer = recordlinkage.Index()

# Block pairing on cuisine_type
indexer.block('cuisine_type')

# Generate pairs
pairs = indexer.index(restaurants, restaurants_new)

In [34]:
# Create a comparison object
comp_cl = recordlinkage.Compare()

# Find exact matches on city, cuisine_types - 
comp_cl.exact('city', 'city', label='city')
comp_cl.exact('cuisine_type', 'cuisine_type', label='cuisine_type')

# Find similar matches of rest_name
comp_cl.string('rest_name', 'rest_name', label='name', threshold = 0.8) 

# Get potential matches and print
potential_matches = comp_cl.compute(pairs, restaurants,restaurants_new)
print(potential_matches)

        city  cuisine_type  name
0   24     0             1   0.0
    63     0             1   0.0
    66     0             1   0.0
1   24     0             1   0.0
    63     0             1   0.0
...      ...           ...   ...
40  18     0             1   0.0
281 18     0             1   0.0
288 18     0             1   0.0
302 18     0             1   0.0
308 18     0             1   0.0

[1302 rows x 3 columns]


In [35]:
# Isolate potential matches with row sum >=3
matches = potential_matches[potential_matches.sum(axis=1) >= 3]

# Get values of second column index of matches
matching_indices = matches.index.get_level_values(1)

# Subset restaurants_new based on non-duplicate values
non_dup = restaurants_new[~restaurants_new.index.isin(matching_indices)]

# Append non_dup to restaurants
full_restaurants = restaurants.append(non_dup)
print(full_restaurants)

     rest_name                  rest_addr               city       phone  \
0      mexican                    mexican            mexican     mexican   
1      mexican                    mexican            mexican     mexican   
2      mexican                    mexican            mexican     mexican   
3      mexican                    mexican            mexican     mexican   
4      mexican                    mexican            mexican     mexican   
..         ...                        ...                ...         ...   
77       feast        1949 westwood blvd.            west la  3104750400   
78    mulberry        17040 ventura blvd.             encino  8189068881   
79  matsuhissa   129 n. la cienega blvd.       beverly hills  3106599639   
80     jiraffe      502 santa monica blvd       santa monica  3109176671   
81    martha's  22nd street grill 25 22nd  st. hermosa beach  3103767786   

   cuisine_type  
0       mexican  
1       mexican  
2       mexican  
3       mexican